In [ ]:
!pip install transformers datasets torch rouge-score

In [ ]:
from rouge_score import rouge_scorer
import json

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
login(token=hf_token)
dataset = load_dataset("vijay-ravichander/cnn_databricks_summarization", split="test")

In [ ]:
def formatting_prompts_func(example):
    messages = [
    {"role": "user", "content": example['instruction'] + "\n" + example["context"] + "\n" + "Given the instruction and context. Generate a summary"},
    {"role": "assistant", "content": example['response']},
    ]
    texts = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
    return { "task" : texts}

dataset = dataset.map(formatting_prompts_func, batched= False)

In [ ]:
def tokenize_function(example):
    outputs = tokenizer(example["task"], truncation=True, padding="max_length", max_length=5096)
    return outputs
    
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns = ["instruction", "context", "response" ], load_from_cache_file=False)
tokenized_datasets.set_format("torch")

In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

output_file = "llama1b_all_r128_cnn_responses.jsonl"
#output_file = "llama1b_attn_r128_cnn_responses.jsonl"
#output_file = "llama1b_mlp_r128_cnn_responses.jsonl"

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'vijay-ravichander/Llama-1B-Summarization-LoRA-All-r128-merged',
    #model_name = 'vijay-ravichander/Llama-1B-Summarization-LoRA-Attn-r128-merged',
    #model_name = 'vijay-ravichander/Llama-1B-Summarization-LoRA-MLP-r128-merged',
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model)

In [ ]:
def generate_summaries(article, num_completions=5):
    summaries = []
    input_prompt = [
        {"role": "user", "content": f"Summarize the following article:\n {article}"
     ]
    input_ids = tokenizer(input_prompt, return_tensors="pt", truncation=True, max_length=1024).input_ids.to(model.device)

    for _ in range(num_completions):
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=128,
            num_return_sequences=1,
            temperature=1.5,
            top_p=0.9,
            do_sample=True
        )
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [ ]:
def compute_rouge_scores(reference_summary, generated_summaries):
    rouge_scores = []
    for summary in generated_summaries:
        scores = scorer.score(reference_summary, summary)
        rouge_scores.append({
            "rouge1": scores["rouge1"].fmeasure,
            "rouge2": scores["rouge2"].fmeasure,
            "rougeL": scores["rougeL"].fmeasure
        })
    return rouge_scores

In [ ]:
with open(output_file, "w") as f:
    for idx, example in enumerate(dataset):
        article = example["article"]
        reference_summary = example["highlights"]

        generated_summaries = generate_summaries(article, num_completions)
        rouge_scores = compute_rouge_scores(reference_summary, generated_summaries)

        result = {
            "example_id": idx,
            "article": article,
            "reference_summary": reference_summary,
            "generated_summaries": generated_summaries,
            "rouge_scores": rouge_scores
        }
        f.write(json.dumps(result) + "\n")
        print(f"Processed example {idx + 1}")

print(f"Results saved to {output_file}")